# Proyek Prediksi Risiko Diabetes dengan Random Forest

Notebook ini bertujuan untuk membangun model Machine Learning yang dapat memprediksi risiko diabetes dengan akurasi tinggi (Training 100%).

### Langkah Utama:
1. **Imputasi Data**: Mengganti nilai 0 yang tidak logis pada kolom medis dengan nilai rata-rata.
2. **Pemisahan Data**: Memisahkan fitur (X) dan target (Y).
3. **Standarisasi**: Menyamakan skala data.
4. **Modeling**: Menggunakan **Random Forest Classifier** untuk performa maksimal.

## 1. Import Library
Kita mengimpor library penting untuk pengolahan data (`pandas`, `numpy`) dan machine learning (`sklearn`).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## 2. Memuat Dataset
Membaca file `diabetes.csv` ke dalam DataFrame Pandas. Pastikan file ini sudah diunggah di direktori yang sama.

In [ ]:
dataset_path = 'diabetes.csv'
diabetes_dataset = pd.read_csv(dataset_path)

print(f"Ukuran data awal: {diabetes_dataset.shape}")
diabetes_dataset.head()

## 3. Preprocessing: Handling Missing Values

Banyak kolom medis seperti `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin`, dan `BMI` memiliki nilai **0**, yang secara medis tidak mungkin (invalid).

**Strategi Perbaikan:**
1. Ubah nilai `0` menjadi `NaN` (Not a Number).
2. Isi `NaN` tersebut dengan nilai **Rata-rata (Mean)** dari masing-masing kolom.

In [ ]:
cols_to_fix = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# 1. Ganti 0 dengan NaN
diabetes_dataset[cols_to_fix] = diabetes_dataset[cols_to_fix].replace(0, np.nan)

# 2. Isi NaN dengan Mean (Rata-rata)
for col in cols_to_fix:
    mean_val = diabetes_dataset[col].mean(skipna=True)
    diabetes_dataset[col] = diabetes_dataset[col].fillna(mean_val)
    print(f"Mengisi nilai kosong di kolom '{col}' dengan rata-rata: {mean_val:.2f}")

print("\nCek data setelah imputasi:")
diabetes_dataset.head()

## 4. Pemisahan Fitur (X) dan Target (Y)

Tahap ini krusial. Kita harus memisahkan antara data yang menjadi **petunjuk** (Features) dan data yang menjadi **kunci jawaban** (Target).

### Penjelasan Code:
- `X = diabetes_dataset.drop(columns='Outcome', axis=1)`:
  - Kita mengambil semua kolom **kecuali** 'Outcome'.
  - Data ini (Pregnancies, Glucose, dst) akan menjadi input bagi model untuk belajar memprediksi.
  - `axis=1` menandakan kita membuang kolom (bukan baris).

- `Y = diabetes_dataset['Outcome']`:
  - Kita hanya mengambil kolom 'Outcome'.
  - Ini adalah label yang berisi **0 (Sehat)** atau **1 (Diabetes)**.
  - Model akan mencoba mencocokkan pola di X agar sesuai dengan Y.

In [ ]:
X = diabetes_dataset.drop(columns='Outcome', axis=1)
Y = diabetes_dataset['Outcome']

print("X (Fitur):")
print(X.head())
print("\nY (Target):")
print(Y.head())

## 5. Standarisasi Data

Algoritma Machine Learning bekerja lebih optimal jika semua fitur memiliki skala yang seragam (rata-rata 0, deviasi standar 1). Kita menggunakan `StandardScaler`.

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
standarized_data = scaler.transform(X)

X = standarized_data
print("Data setelah standarisasi (5 baris pertama):")
print(X[:5])

## 6. Train-Test Split

Membagi dataset menjadi:
- **Training Set (80%)**: Untuk melatih model.
- **Testing Set (20%)**: Untuk menguji seberapa pintar model kita pada data baru.

`stratify=Y` digunakan agar perbandingan jumlah pasien diabetes dan non-diabetes tetap seimbang di kedua set.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2, stratify=Y)

print(f"Jumlah Data Training: {X_train.shape[0]}")
print(f"Jumlah Data Testing : {X_test.shape[0]}")

## 7. Melatih Model (Random Forest)

Untuk mencapai **Akurasi Training 100%**, kita menggunakan **Random Forest Classifier**.

### Mengapa Random Forest?
- Random Forest terdiri dari banyak *Decision Trees* (Pohon Keputusan).
- Algoritma ini sangat powerful dan mampu mempelajari pola data training dengan sangat detail (bahkan sampai menghafal noise jika tidak dibatasi).
- Ini lebih kuat dibandingkan Linear SVM yang hanya menarik garis lurus pemisah.

In [ ]:
# n_estimators=100 artinya kita menggunakan 100 pohon keputusan.
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, Y_train)

## 8. Evaluasi Akurasi

Kita cek akurasi model pada data training dan testing.

In [ ]:
# Evaluasi Training Data
X_train_prediction = classifier.predict(X_train)
train_acc = accuracy_score(X_train_prediction, Y_train)
print(f'Akurasi Training: {train_acc * 100:.2f}%')

# Evaluasi Testing Data
X_test_prediction = classifier.predict(X_test)
test_acc = accuracy_score(X_test_prediction, Y_test)
print(f'Akurasi Testing : {test_acc * 100:.2f}%')

## 9. Simulasi Prediksi
Mencoba memprediksi satu pasien baru.

In [ ]:
input_data = (5, 166, 72, 19, 175, 25.8, 0.587, 51)

# Ubah ke DataFrame agar sesuai dengan fitur saat training
feature_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
input_df = pd.DataFrame([input_data], columns=feature_names)

# Standarisasi
std_data = scaler.transform(input_df)

# Prediksi
prediction = classifier.predict(std_data)

print(f"Data Input: {input_data}")
if prediction[0] == 0:
    print('Hasil: Pasien TIDAK Berisiko Diabetes')
else:
    print('Hasil: Pasien BERISIKO Diabetes')